## Random Forest Regression - Salaries

In [1]:
# Import libraries and dependencies
import pandas as pd
from pathlib import Path
# from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor # Note Regressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import streamlit as st

In [2]:
# Extra imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.feature_selection import RFE
from sklearn.gaussian_process import GaussianProcessRegressor

## Philip code

In [3]:
# Philip code
def clean_categories(categories, cutoff):
    category_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            category_map[categories.index[i]] = categories.index[i]
        else:
            category_map[categories.index[i]] = 'Other'
    return category_map

# @st.cache_data
def load_data():
    # Read data
    salary_df = pd.read_csv("./ds_salaries.csv")

    # Clean data
    salary_df = salary_df.drop(['salary', 'salary_currency', 'employee_residence'], axis=1)
    salary_df = salary_df[salary_df["salary_in_usd"].notnull()]
    salary_df = salary_df.dropna()

    values = ['FT']
    salary_df = salary_df[salary_df['employment_type'].isin(values)]

    # Clean company location column data
    company_location_map = clean_categories(salary_df['company_location'].value_counts(), 100) # What if we changed it to >35?
    salary_df['company_location'] = salary_df['company_location'].map(company_location_map)
    salary_df = salary_df[salary_df['salary_in_usd'] <= 175000]
    salary_df = salary_df[salary_df['salary_in_usd'] >= 25000]

    # Clean job title column data
    job_title_map = clean_categories(salary_df['job_title'].value_counts(), 100)
    salary_df['job_title'] = salary_df['job_title'].map(job_title_map)
    return salary_df

In [4]:
# Rows in Philip dataset
display(load_data().shape[0])
display(len(load_data()))
display(load_data().head())

2699

2699

,work_year,experience_level,employment_type,job_title,salary_in_usd,remote_ratio,company_location,company_size
0,2023,SE,FT,Other,85847,100,Other,L
3,2023,SE,FT,Data Scientist,175000,100,Other,M
4,2023,SE,FT,Data Scientist,120000,100,Other,M
6,2023,SE,FT,Other,136000,0,US,L
8,2023,SE,FT,Data Scientist,141000,0,Other,M


In [5]:
# Models for Philip datasets
# Set features
y = load_data()["salary_in_usd"]
X = load_data().drop(columns = "salary_in_usd")

# Convert the categorical variables to 0, 1 using get_dummies
X = pd.get_dummies(X)

# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [6]:
# Define the regressors
regressors = [

    ('Random Forest', RandomForestRegressor()),
    ('Gradient Boosting', GradientBoostingRegressor()),
    ('KNN', KNeighborsRegressor()),
    ('Decision Tree', DecisionTreeRegressor()),
    ("Linear Regression", LinearRegression()),
    ("Support Vector", SVR()),
    ("Gaussian Process", GaussianProcessRegressor())
    
]

# Create an empty DataFrame to store the metrics
metrics_df = pd.DataFrame(columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Score'])

# Iterate over each regressor
for reg_name, reg in regressors:
    steps = [('MinMax',StandardScaler()),('Regressor', reg)]
    pipeline = Pipeline(steps)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Calculate the evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = metrics.r2_score(y_test, y_pred)

    new_row = {
        'Model': reg_name,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2 Score': r2
    }

    metrics_df = pd.concat([metrics_df, pd.DataFrame([new_row])], ignore_index=True)

# Sort the metrics DataFrame by RMSE and R2 Score in ascending order
p_sorted_metrics_df = metrics_df.sort_values(by=['MAE'], ascending=[True])

p_sorted_metrics_df

# MSE is analogous to the variance, whereas RMSE is akin to the standard deviation.

,Model,MAE,MSE,RMSE,R2 Score
1,Gradient Boosting,24471.585300,9.117489e+08,30195.180846,0.322670
4,Linear Regression,24568.164964,9.412271e+08,30679.424070,0.300770
0,Random Forest,24573.826085,9.480940e+08,30791.134658,0.295669
2,KNN,25272.637333,9.942059e+08,31531.030787,0.261413
3,Decision Tree,25417.791889,1.031816e+09,32121.898204,0.233472
6,Gaussian Process,27420.233305,1.239070e+09,35200.419946,0.079506
5,Support Vector,30962.486419,1.363934e+09,36931.473264,-0.013255


## End Philip code

In [7]:
# Read data
file_path = Path("./ds_salaries.csv")
df_salaries = pd.read_csv(file_path)
df_salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [8]:
# Too many job titles need to consolidate
df_salaries.nunique()

work_year                4
experience_level         4
employment_type          4
job_title               93
salary                 815
salary_currency         20
salary_in_usd         1035
employee_residence      78
remote_ratio             3
company_location        72
company_size             3
dtype: int64

In [9]:
# Review employee_residence
df_salaries["employee_residence"].value_counts().nlargest(10)

# Consider cutoff at FR

US    3004
GB     167
CA      85
ES      80
IN      71
DE      48
FR      38
PT      18
BR      18
GR      16
Name: employee_residence, dtype: int64

In [10]:
# Review employment_type
df_salaries["employment_type"].value_counts()

FT    3718
PT      17
CT      10
FL      10
Name: employment_type, dtype: int64

In [11]:
df_salaries["company_location"].value_counts().nlargest(15)
# Consider cutoff at FR

US    3040
GB     172
CA      87
ES      77
IN      58
DE      56
FR      34
BR      15
AU      14
GR      14
PT      14
NL      13
MX      10
IE       7
SG       6
Name: company_location, dtype: int64

In [12]:
df_salaries[["experience_level", "job_title", "salary_in_usd"]].sort_values(by="salary_in_usd", ascending=False).head()

,experience_level,job_title,salary_in_usd
3522,MI,Research Scientist,450000
2011,MI,Data Analyst,430967
528,SE,AI Scientist,423834
3747,MI,Applied Machine Learning Scientist,423000
3675,EX,Principal Data Scientist,416000


In [45]:
pd.set_option('display.max_rows', None)
df_salaries["job_title"].value_counts().nlargest()

Data Engineer                1040
Data Scientist                840
Data Analyst                  612
Machine Learning Engineer     289
Analytics Engineer            103
Name: job_title, dtype: int64

## Consider grouping job_title

In [14]:
# def assign_broader_category(job_title):
#     data_engineering = ["Data Engineer", "Data Analyst", "Analytics Engineer", "BI Data Analyst", "Business Data Analyst", "BI Developer", "BI Analyst", "Business Intelligence Engineer", "BI Data Engineer", "Power BI Developer"]
#     data_scientist = ["Data Scientist", "Applied Scientist", "Research Scientist", "3D Computer Vision Researcher", "Deep Learning Researcher", "AI/Computer Vision Engineer"]
#     machine_learning = ["Machine Learning Engineer", "ML Engineer", "Lead Machine Learning Engineer", "Principal Machine Learning Engineer"]
#     data_architecture = ["Data Architect", "Big Data Architect", "Cloud Data Architect", "Principal Data Architect"]
#     management = ["Data Science Manager", "Director of Data Science", "Head of Data Science", "Data Scientist Lead", "Head of Machine Learning", "Manager Data Management", "Data Analytics Manager"]
    
#     if job_title in data_engineering:
#         return "Data Engineering"
#     elif job_title in data_scientist:
#         return "Data Science"
#     elif job_title in machine_learning:
#         return "Machine Learning"
#     elif job_title in data_architecture:
#         return "Data Architecture"
#     elif job_title in management:
#         return "Management"
#     else:
#         return "Other"

# # Apply the function to the 'job_title' column and create a new column 'job_category'
# df['job_category'] = df['job_title'].apply(assign_broader_category)

In [15]:
# Drop salary, salary_currency
df_salaries = df_salaries.drop(columns = ["salary", "salary_currency"])

In [16]:
# Set features
y = df_salaries["salary_in_usd"]
X = df_salaries.drop(columns = "salary_in_usd")

In [17]:
# Convert the categorical variables to 0, 1 using get_dummies
X = pd.get_dummies(X)

In [18]:
# Review the categorical variables
X.head()

,work_year,remote_ratio,experience_level_EN,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,...,company_location_SI,company_location_SK,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_size_L,company_size_M,company_size_S
0,2023,100,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,2023,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,2023,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,2023,100,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,2023,100,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [20]:
# Note: Scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Create a random forest classifier
rf_model = RandomForestRegressor(random_state=1)

In [22]:
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [23]:
predictions = rf_model.predict(X_test)

In [24]:
# # Santiago code
# import numpy as np
# candidate = pd.DataFrame(data = np.zeros((1,261)),columns = X_test.columns) # turns all values to zeros
# candidate['company_size_L'] = 1
# candidate['work_year_2023'] = 1
# candidate['experience_level_SE'] = 1
# rf_model.predict(candidate)

In [25]:
y_pred = rf_model.predict(X_test)

results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y_test
}).reset_index(drop=True)

results

,Prediction,Actual
0,157164.044323,130000
1,157164.044323,185900
2,86346.521517,61566
3,177542.242884,297300
4,99924.763333,50000
...,...,...
934,171108.137270,260000
935,118278.700000,127221
936,70387.128243,73546
937,171108.137270,183000


In [26]:
importances = rf_model.feature_importances_
sorted(zip(importances, X.columns), reverse = True)[:20]

# Create a dataframe of the important features
importances_df = pd.DataFrame(sorted(zip(importances, X.columns), reverse = True)[:])

# Rename the columns
importances_df = importances_df.rename(columns={0: 'Importance', 1: 'Feature'})

# Set the index
importances_df = importances_df.set_index('Feature')

# Sort the dataframe by feature importance
importances_df = importances_df.sort_values(by='Importance',ascending=False)

In [37]:
display(importances_df.head(5))
display(importances_df.tail(5))

,Importance
Feature,
employee_residence_US,0.377367
job_title_Data Analyst,0.084665
work_year,0.045695
experience_level_SE,0.045121
experience_level_EN,0.042843


,Importance
Feature,
employee_residence_SK,0.0
job_title_Compliance Data Analyst,0.0
job_title_Power BI Developer,0.0
job_title_Principal Data Architect,0.0
company_location_AL,0.0


In [38]:
# Display dataframes side by side
import numpy as np
import pandas as pd   
from IPython.display import display_html 

df1 = importances_df.head(5)
df2 = importances_df.tail(5
                         )

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('Caption table 1')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('Caption table 2')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,Importance
Feature,
employee_residence_US,0.377367
job_title_Data Analyst,0.084665
work_year,0.045695
experience_level_SE,0.045121
experience_level_EN,0.042843
,Importance
Feature,
employee_residence_SK,0.000000
job_title_Compliance Data Analyst,0.000000


## Data Analysis

In [29]:
# Source: https://www.kaggle.com/code/wilfreddj/simple-salaries-regression

In [30]:
# Define the regressors
regressors = [

    ('Random Forest', RandomForestRegressor()),
    ('Gradient Boosting', GradientBoostingRegressor()),
    ('KNN', KNeighborsRegressor()),
    ('Decision Tree', DecisionTreeRegressor()),
    ("Linear Regression", LinearRegression()),
    ("Support Vector", SVR()),
    ("Gaussian Process", GaussianProcessRegressor())
    
]

# Create an empty DataFrame to store the metrics
metrics_df = pd.DataFrame(columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Score'])

# Iterate over each regressor
for reg_name, reg in regressors:
    steps = [('MinMax',StandardScaler()),('Regressor', reg)]
    pipeline = Pipeline(steps)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Calculate the evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = metrics.r2_score(y_test, y_pred)

    new_row = {
        'Model': reg_name,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2 Score': r2
    }

    metrics_df = pd.concat([metrics_df, pd.DataFrame([new_row])], ignore_index=True)

# Sort the metrics DataFrame by RMSE and R2 Score in ascending order
sorted_metrics_df = metrics_df.sort_values(by=['MAE', 'R2 Score'], ascending=[True, False])

sorted_metrics_df

# MSE is analogous to the variance, whereas RMSE is akin to the standard deviation.

,Model,MAE,MSE,RMSE,R2 Score
0,Random Forest,3.600103e+04,2.459757e+09,4.959594e+04,4.002516e-01
1,Gradient Boosting,3.665919e+04,2.435962e+09,4.935547e+04,4.060534e-01
3,Decision Tree,3.750913e+04,2.681873e+09,5.178680e+04,3.460945e-01
2,KNN,4.102534e+04,3.109481e+09,5.576272e+04,2.418331e-01
6,Gaussian Process,4.342072e+04,3.721759e+09,6.100622e+04,9.254507e-02
5,Support Vector,4.922661e+04,4.102277e+09,6.404902e+04,-2.344148e-04
4,Linear Regression,8.752684e+16,1.044955e+36,1.022231e+18,-2.547854e+26


In [31]:
df1 = sorted_metrics_df
df2 = p_sorted_metrics_df

df2

,Model,MAE,MSE,RMSE,R2 Score
1,Gradient Boosting,24471.585300,9.117489e+08,30195.180846,0.322670
4,Linear Regression,24568.164964,9.412271e+08,30679.424070,0.300770
0,Random Forest,24573.826085,9.480940e+08,30791.134658,0.295669
2,KNN,25272.637333,9.942059e+08,31531.030787,0.261413
3,Decision Tree,25417.791889,1.031816e+09,32121.898204,0.233472
6,Gaussian Process,27420.233305,1.239070e+09,35200.419946,0.079506
5,Support Vector,30962.486419,1.363934e+09,36931.473264,-0.013255


## Feature Importance / Correlation

In [32]:
# Concat the processed dummy features and the target variable
processed_df = pd.concat([X, y], axis=1)
processed_df.head()

,work_year,remote_ratio,experience_level_EN,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,...,company_location_SK,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_size_L,company_size_M,company_size_S,salary_in_usd
0,2023,100,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,85847
1,2023,100,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,30000
2,2023,100,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,25500
3,2023,100,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,175000
4,2023,100,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,120000


In [33]:
corr = processed_df.corr()
target_corr = corr['salary_in_usd'].drop('salary_in_usd')

# Sort correlation values in descending order
target_corr_sorted = target_corr.sort_values(ascending=False)[:20]

pd.DataFrame(target_corr_sorted)
# sns.set(font_scale=0.8)
# sns.set_style("white")
# sns.set_palette("PuBuGn_d")
# sns.heatmap(target_corr_sorted.to_frame(), cmap="rocket", annot=True, fmt='.2f')
# plt.title('Correlation with Salary')
# plt.show()

,salary_in_usd
employee_residence_US,0.483816
company_location_US,0.466103
experience_level_SE,0.349900
work_year,0.228290
company_size_M,0.201830
experience_level_EX,0.160986
employment_type_FT,0.118263
job_title_Data Science Manager,0.106700
job_title_Applied Scientist,0.104685
job_title_Machine Learning Engineer,0.078412


In [43]:
pd.DataFrame(target_corr.sort_values(ascending=False))

,salary_in_usd
employee_residence_US,0.483816
company_location_US,0.466103
experience_level_SE,0.349900
work_year,0.228290
company_size_M,0.201830
...,...
job_title_Data Analyst,-0.201951
company_location_IN,-0.213313
employee_residence_IN,-0.223170
experience_level_MI,-0.273791


In [ ]:


# Sort correlation values in descending order
target_corr_sorted = target_corr.sort_values(ascending=False)[:20]

sns.set(font_scale=0.8)
sns.set_style("white")
sns.set_palette("PuBuGn_d")
sns.heatmap(target_corr_sorted.to_frame(), cmap="rocket", annot=True, fmt='.2f')
plt.title('Correlation with Salary')
plt.show()

In [ ]:
# Note: Need to condence the job_titles

In [ ]:
importances_df
df = pd.DataFrame(importances_df.values)

In [ ]:
df 

In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_

In [ ]:
# Zip the feature importances with the associated feature name
important_features = zip(X.columns,rf_model.feature_importances_)

important_features

In [ ]:
# Create a dataframe of the important features
importances_df = pd.DataFrame(important_features)

# Rename the columns
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

# Set the index
importances_df = importances_df.set_index('Feature')

# Sort the dataframe by feature importance
importances_df = importances_df.sort_values(by='Importance',ascending=True)

In [ ]:
# Plot the top 10 most important features
importances_df[0:10].plot(
    kind='barh', 
    color='lightgreen', 
    title= 'Feature Importance', 
    legend=True)

In [ ]:
importances_df.head()